In [ ]:
import pandas as pd

In [24]:
stores = pd.read_csv('data/stores.csv')
train = pd.read_csv('data/train.csv')
holidays = pd.read_csv('data/holidays_events.csv')
oil = pd.read_csv('data/oil.csv')

In [25]:
stores = stores.drop(columns=['type', 'cluster'])
holidays = holidays[holidays['transferred'] == False]
holidays = holidays.drop(columns=['transferred', 'description'])

holidays['date'] = pd.to_datetime(holidays['date'])
train['date'] = pd.to_datetime(train['date'])
oil['date'] = pd.to_datetime(oil['date'])

In [26]:
train_and_stores = pd.merge(train, stores, how='left', on='store_nbr')
train_and_stores_and_holidays = pd.merge(train_and_stores, holidays, how='left', on='date')

# Add a new column to indicate if the date is a holiday and set true if locale is National
train_and_stores_and_holidays['is_holiday'] = train_and_stores_and_holidays['locale'] == 'National'

# Set true if locale is regional and state is equal to locale_name
train_and_stores_and_holidays['is_holiday'] = train_and_stores_and_holidays['is_holiday'] | (
    (train_and_stores_and_holidays['locale'] == 'Regional') & 
    (train_and_stores_and_holidays['state'] == train_and_stores_and_holidays['locale_name'])
)

# Set true if locale is local and city is equal to locale_name
train_and_stores_and_holidays['is_holiday'] = train_and_stores_and_holidays['is_holiday'] | (
    (train_and_stores_and_holidays['locale'] == 'Local') & 
    (train_and_stores_and_holidays['city'] == train_and_stores_and_holidays['locale_name'])
)

# Drop the columns that are not needed
train_and_stores_and_holidays.drop(columns=['locale', 'locale_name', 'state', 'city'], inplace=True)

display(train_and_stores_and_holidays.head(5))

,id,date,store_nbr,family,sales,onpromotion,type,is_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,True
1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,True
2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,True
3,3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,True
4,4,2013-01-01,1,BOOKS,0.0,0,Holiday,True


In [ ]:
add_oil = pd.merge(train_and_stores_and_holidays, oil, how='left', on='date')
add_oil['dcoilwtico'] = add_oil['dcoilwtico'].bfill()

,id,date,store_nbr,family,sales,onpromotion,type,is_holiday,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Holiday,True,93.14
1,1,2013-01-01,1,BABY CARE,0.000,0,Holiday,True,93.14
2,2,2013-01-01,1,BEAUTY,0.000,0,Holiday,True,93.14
3,3,2013-01-01,1,BEVERAGES,0.000,0,Holiday,True,93.14
4,4,2013-01-01,1,BOOKS,0.000,0,Holiday,True,93.14
...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,False,47.57
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,False,47.57
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,False,47.57
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,False,47.57


In [ ]:
df = add_oil.copy()
df = df.drop(columns=['type'])

# Save the data to a CSV file
df.to_csv('train.csv', index=False)